- https://keras.io/layers/embeddings/
- https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
import tensorflow_hub as hub


/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model
embedding = loadGloveModel('/Users/dweepa/Downloads/glove.6B.50d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [3]:
TRAIN_CSV = '../train.csv'
TEST_CSV = '../test.csv'

In [4]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [40]:
def pad(A, length):
    arr = np.zeros(length)
    arr[:len(A)] = A
    return arr
def vec(sentence):
    l = []
    
    for word in sentence.lower().split():
        if word not in embedding.keys():
            embedding[word] = np.zeros(50)
        l = l+ list(embedding[word])
    return pad(np.asarray(l), 50*20)

In [41]:
vec("What how are you$").shape

(1000,)

In [42]:
# Iterate over the questions only of both training and test datasets
questions_cols = ['question1', 'question2']
trainvec=[]
testvec=[]
count = 0
for sentence in zip(train_df['question1'],train_df['question2']):
    if count>20:
        break
    try:
        trainvec.append([vec(sentence[0]),vec(sentence[1])])
    except:
        continue
    count+=1
count = 0
for sentence in zip(test_df['question1'],test_df['question2']):
    if count>20:
        break
    try:
        testvec.append([vec(sentence[0]),vec(sentence[1])])
    except:
        continue
    count+=1

In [43]:
max_seq_length = max(trainvec[:0],trainvec[:1])

# Split to train validation
validation_size = 40000
training_size = len(train_df) - validation_size

X = trainvec
Y = train_df['is_duplicate']
left = [row[0] for row in X]
right = [row[1] for row in X]

left_t = [row[0] for row in testvec]
right_t = [row[1] for row in testvec]
len(right)

21

In [30]:
# Split to dicts
X_train = {'left': left, 'right': right}
X_test = {'left': left_t, 'right': right_t}

# Convert labels to their numpy representations
Y = Y.values
'''
# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)
'''


"\n# Zero padding\nfor dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):\n    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)\n\n# Make sure everything is ok\nassert X_train['left'].shape == X_train['right'].shape\nassert len(X_train['left']) == len(Y_train)\n"

In [58]:
x_left = [a[0] for a in X]
x_right = [a[1] for a in X]

In [67]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

ileft = Input(shape=[1000, ])
iright = Input(shape=[1000, ])

shared_lstm = LSTM(n_hidden)
left_output = shared_lstm(ileft)
right_output = shared_lstm(iright)

malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

malstm = Model([ileft, iright], [malstm_distance])

optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

training_start_time = time()

malstm_trained = malstm.fit([x_left,x_right], Y, batch_size=batch_size, nb_epoch=n_epoch)

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

ValueError: Input 0 is incompatible with layer lstm_16: expected ndim=3, found ndim=2